In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using LinearAlgebra
using Plots
using ForwardDiff
using RobotZoo
using RobotDynamics
using Ipopt
using MathOptInterface
using StaticArrays
using SparseArrays
using MATLAB 
using CSV
using DataFrames
# Pkg.add("Statistics")
using Statistics 
include("Dimensions.jl")
include("Calculate_LPoints.jl")
include("Non_lin_exp.jl")
include("CR3BP_Dynamics.jl")
include("sparseblocks.jl")
include("MOI.jl")
include("NLP.jl")
const MOI = MathOptInterface;

In [ ]:
# Define states and controls
Nx = 6     # Number of states
Nu = 1     # Number of controls

# Specify System, Lagrange point, and if we want dimensional or dimensionless system
LagrangePoint = 1
Dimensionless = true
System = "Sun-Earth"

# Calculate Lagrange points for specified system
L1, L2, L3, L4, L5 = get_LPoints(System)

# Get period of orbit, define number of knot points and generate thist vector
Tfinal, Tfinal_days = get_Period(LagrangePoint, System)
Nt = 50              # Number of knot points --> generates number of time steps-52
h = Tfinal/(Nt - 1)  # Timestep

thist = Array(range(0, Tfinal, step = h)); 
n_nlp = (Nx + Nu)*Nt # Number of decision variables-350
m_nlp = Nx*(Nt)  # 306
u_idx = [(Nx + Nu)*(i - 1) .+ (Nx + 1:Nx + Nu) for i = 1:Nt];
x_idx = [(Nx + Nu)*(i - 1) .+ (1:Nx) for i = 1:Nt];

println("Timestep h: ", h)
println("Period (dimensionless): ", Tfinal)
println("Period (in days): ", Tfinal_days)


In [ ]:
# Generate Reference Trajectory 
Xref,Uref = get_reference()
print("Step size in x: ", Xref[2][1] - Xref[1][1])


In [ ]:
xRef = zeros(Nt)
yRef = zeros(Nt)
zRef = zeros(Nt)

for k = 1:Nt
    xRef[k] = Xref[k][1]
    yRef[k] = Xref[k][2]
    zRef[k] = Xref[k][3]
end

# Plots.plot(xRef,yRef)

In [ ]:
# Cost weights
Q = Diagonal([0.95*ones(3); 0.95*ones(3)]);
R = 0.001;

In [ ]:
function stage_cost(x,u,Xref,Uref)
    return 0.5*((x.-Xref)'*Q*(x.-Xref)) + 0.5*(u.-Uref)'*R*(u.-Uref)
end

In [ ]:
function cost(ztraj)
    z = reshape(ztraj,Nx+Nu,Nt)
    J = 0.0
    for k = 1:Nt
        J += stage_cost(z[1:Nx,k],z[(Nx+1):(Nx+Nu),k],Xref[k],Uref[k])
    end
    return J
end

In [ ]:
struct ProblemMOI <: MOI.AbstractNLPEvaluator
    n_nlp::Int
    m_nlp::Int
    idx_ineq
    obj_grad::Bool
    con_jac::Bool
    sparsity_jac
    sparsity_hess
    primal_bounds
    constraint_bounds
    hessian_lagrangian::Bool
    blocks::BlockViews
end 

function ProblemMOI(n_nlp,m_nlp;
    idx_ineq=(1:0),
    obj_grad=true,
    con_jac=true,
    sparsity_jac=sparsity_jacobian(n_nlp,m_nlp),
    sparsity_hess=sparsity_hessian(n_nlp,m_nlp),
    primal_bounds=primal_bounds(n_nlp),
    constraint_bounds=constraint_bounds(m_nlp,idx_ineq=idx_ineq),
    hessian_lagrangian=false,
    blocks = BlockViews(m_nlp,n_nlp))
    ProblemMOI(n_nlp,m_nlp,
            idx_ineq,
            obj_grad,
            con_jac,
            sparsity_jac,
            sparsity_hess,
            primal_bounds,
            constraint_bounds,
            hessian_lagrangian,
            blocks)
end

In [ ]:
#Guess
z0 = zeros((Nx+Nu)*(Nt))
z0[x_idx[1]] = Xref[1]
for k=1:Nt
    z0[x_idx[k]] = Xref[k]
    z0[u_idx[k]] .= Uref[k]
end

In [ ]:
prob = ProblemMOI(n_nlp,m_nlp)
z_sol = solve(z0,prob) # solve

In [ ]:
ztraj = reshape(z_sol,Nx+Nu,Nt);
xtraj = ztraj[1:Nx,:];#6 x 50
utraj = ztraj[(Nx+1):(Nx+Nu),:]; # 1 x 91

In [ ]:
# Plot reference vs IPOPT orbit
p = Plots.plot(xRef, yRef, label = "Reference")
Plots.plot!(p, xtraj[1,:], xtraj[2,:], label = "IPOPT")

In [ ]:
# Plot X, Y, and Z individually
reduceBy = findmax(xtraj[1,:])
p = Plots.plot(thist, xtraj[1,:] .- reduceBy[1], label = "X")
Plots.plot!(p, thist, xtraj[2,:], label = "Y")
Plots.plot!(p, thist, xtraj[3,:], label = "Z")

In [ ]:
# Look at period generated by IPOPT
u = 0
for i=1:Nt
    u += utraj[1, i]
end
Period_IPOPT = (u/(2*pi))*3.1536e7/(24*60*60)
@show Period_IPOPT

# Show plot of timesteps
mx, mi = findmax(utraj[1,:])
# Plots.plot(thist,utraj[1,:], label = "Timesteps")
Plots.scatter(thist,utraj[1,:], label = "Timesteps")

In [ ]:
xTraj = zeros(Nt)
yTraj = zeros(Nt)
ztraj = zeros(Nt)

for k = 1:Nt
    xTraj[k] = xtraj[1,k]
    yTraj[k] = xtraj[2,k]
    ztraj[k] = xtraj[3,k]
end


In [ ]:
# mat"""
# plot3($xTraj,$yTraj,$ztraj,'DisplayName','IPOPT')
# hold on
# plot3($xRef,$yRef,$zRef,'DisplayName','Reference')
# plot3($xTraj($mi),$yTraj($mi),$ztraj($mi),'ro')
# grid on
# hold off
# legend('IPOPT','Reference')
# """ 

In [ ]:
# mat"""
# plot($xTraj,$yTraj,'DisplayName','IPOPT')
# hold on
# plot($xRef,$yRef,'DisplayName','Reference')
# grid on
# hold off
# legend('IPOPT','Reference')
# """

In [ ]:
### Implement Continuation of Libration point Orbits
include("Families.jl")
numIters = 10
scaleValue = 0.05
Up = true
x_fam = Generating_Orbit_Families(z0, numIters, scaleValue, Up)



In [ ]:
L = size(x_fam)[3]
mat"""
plot3($xRef,$yRef,$zRef,'k-','DisplayName','Reference')
hold on
grid on
plot3($x_fam(1,:,1),$x_fam(2,:,1),$x_fam(3,:,1),'r-','DisplayName','IPOPT Results')
for i = 2:$L
    plot3($x_fam(1,:,i),$x_fam(2,:,i),$x_fam(3,:,i),'r-')
end
hold off
title('Family of Orbits around L1 (200 iterations, scaling by 5%)', 'FontSize', 24)
xlabel('X', 'FontSize', 20)
ylabel('Y', 'FontSize', 20)
zlabel('Z', 'FontSize', 20)
legend('Reference','IPOPT Results', 'FontSize', 16)
""" 



In [ ]:
# #make sure you use dimensionless
# Nu = 3
# #Establish Variables
# Q = Diagonal([10,10,10,10,10,10])
# R = Diagonal([1,1,1]) #we want to conserve fuel so penalizing the controls takes priority
# Qf = Diagonal([100,100,100,100,100,100]) #this can be the same as Q since its a periodic orbit so the end isnt special
# K = [zeros(Nu,Nx)*NaN for k = 1:Nt-1]
# P = [zeros(Nx,Nx)*NaN for k = 1:Nt]

# #Initialize jacobians
# A = [zeros(Nx,Nx) for k = 1:Nt]
# B = [zeros(Nu,Nu) for k = 1:Nt-1]

# #Initialize control
# U = [zeros(Nu) for k = 1:Nt-1] #under ideal circumstances, no controls need to be expended. (does this mean B will stay zeros?)

# ###############################
# #ACTUALLY RUN THE TVLQR

# #xtraj is our halo orbit trajectory we are trying to stick to
# #Xsat is the satellite
# @show size(xtraj), size(A[1]), size(B[1]),size(U),size(utraj)
# # linearize!(A,B,xtraj, U)

# # calc_gains!(xtraj, A, B, P, K, Qf, Q)

# # Xsat = [zeros(Nx) for k = 1:Nt] 

# # #i still need to add some minor purturbation to this or the controller should jst do nothing
# # #I'll either do it at the beginning or at some random timestep(s)
# # Xsat[1] = copy(xtraj[1])*1.0001
# # # Xsat[1] = Xsat[1] + [.0001, .0001, .0001, 0, 0, 0]

# # for k = 1:Nt-1
# #     U[k] = U[k] - K[k]*(Xsat[k]-xtraj[k]) #in hw this was often the get_control function
# #     Xsat[k+1] = rk4WithU(CR3BPdynamicsWithUforA, Xsat[k], U[k], utraj[k])  #discrete_dynamics(RK4, model, X[k], U[k], times[k], dt) 
# # end

# # ###############################



# # #plot error
# # sse = zeros(length(Xsat))
# # for k = 1:Nt
# #     sse[k] = norm(Xsat[k]-xtraj[k])
# # end
# # t = Array(range(1, Nt, step = 1));

# # xtrajX = zeros(length(xtraj))
# # xtrajY = zeros(length(xtraj))
# # xtrajZ = zeros(length(xtraj))

# # XsatX = zeros(length(xtraj))
# # XsatY = zeros(length(xtraj))
# # XsatZ = zeros(length(xtraj))

# # for k in 1:Nt
# #     xtrajX[k] = xtraj[k][1]
# #     xtrajY[k] = xtraj[k][2]
# #     xtrajZ[k] = xtraj[k][3]
    
# #     XsatX[k] = Xsat[k][1]
# #     XsatY[k] = Xsat[k][2]
# #     XsatZ[k] = Xsat[k][3]
# # end


# # Ux = zeros(length(U))
# # Uy = zeros(length(U))
# # Uz = zeros(length(U))
# # for k in 1:Nt-1
# #     Ux[k] = U[k][1]
# #     Uy[k] = U[k][2]
# #     Uz[k] = U[k][3]
# # end

# # print(sse[49])
# # plot(t, sse)
# # # plot(t[1:end-1], Uz)
# # # plot(xtraj, Xsat)
# # # print(U)

# # ##############################################

# # p = Plots.plot(xtrajX, xtrajY, label = "IPOPT")
# # Plots.plot!(p, XsatX, XsatY, label = "TVLQR")

# # ###############################################
# # p = Plots.plot(t[1:end-1], Ux, label = "Ux")
# # Plots.plot!(p, t[1:end-1], Uy, label = "Uy")
# # Plots.plot!(p, t[1:end-1], Uz, label = "Uz")

# # ###############################################

# # p = Plots.plot3d(XsatX,XsatY,XsatZ, label = "Xsat")
# # Plots.plot3d!(p, xtrajX,xtrajY,xtrajZ, label = "xtraj")
# # # plot3d(XsatX,XsatY,XsatZ)

# #############################################